# Informe - Trabajo Práctico N°2 - Estructuras de Datos - 2019

Integrantes: Quintana Axel, Bellesi Nicolás

El proyecto entregado consiste de la implementación de un software encargado de recolectar noticias de la web, publicadas por distintos diarios argentinos, y posteriormente creando un indice que permite realizar busquedas dentro de las noticias recopiladas. En nuestro caso utilizamos los diarios Clarín, La Voz, Télam, Ámbito Financiero y Perfil.



Diseño del trabajo:
    
Consta de 8 clases para el sistema, 4 para tests y una para excepciones. El programa recolecta las noticias de los medios especificados en el archivo de configuración .ini, cada un cierto tiempo especificado tambien en dicho archivo y las guarda en una carpeta .xml. Posteriormente se procesan dichas noticias y se genera un índice invertido utilizando el algoritmo BSBI (Block Sorted Based Indexing).
Con el índice ya creado, se pueden realizar búsquedas booleanas AND y OR, la primera mostrara las noticias donde se encuentren TODAS las palabras claves ingresadas, la segunda mostrará una lista de noticias para cada palabra.
Finalmente, esta la posibilidad de comprimir la lista de apariciones de cada término, utilizando una codificación de longitud variable, para luego generar un índice invertido comprimido (en este índice también pueden realizarse búsquedas).

A continuación se detalla la funcionalidad de cada clase del proyecto:
    

- config.py: esta clase se encarga de leer el archivo de configuracion .ini y generar un diccionario que incluye los nombres de cada medio, sección y los links de sus rss, utlizando la libreria ConfigParser. Tambien se crearán algunos diccionarios auxiliares que se utilizarán a lo largo de la ejecucion del programa.


- newsCollector.py: realiza la recolección de noticias. Primero con el método crearXMLmedios ,utilizando el diccionario previamente creado , se descarga del  rss del medio y sección pasado por parámetro utilizando la libreria Requests, las noticias disponibles ese día y crea  un xml que indica a cual corresponde. Este método se ejecuta en obtenerNoticias con un doble for que recorre el diccionario de config y crea todos los medios + sección de los que se tenga información. El método eliminarNoticiasRepetidas , recibe 2 listas por parámetro , 1 es la de las últimas noticias descargadas y la otra nuestro cuerpo de noticias local, con estás listas comprara los títulos de cada elemento y elimina de la lista descargada los que ya estén en la lista local. El método crearFecha a partir del pubdate leído con xpath lo acomoda y crea la fecha con date Ámbito y la La-voz usan crearFecha2 porque el pubdate lo presentan  de una manera distinta. El método actualizarXML , sería el método principal , encargado de leer las noticias que descargamos y las que tenemos guardadas y juntarlas,verificando primero si hay repetidos , es decir que ya están en el cuerpo , y  comparando la fecha de publicación de cada noticia para agregarla al cuerpo de noticias  manteniendo el orden . Dentro de este método se ejecutan 2 métodos , IDnoticias que agrega un ID como atributo a cada item de cada xml y numerarMedioYSeccion que agrega en descripción el número de medio y de sección correspondiente a cada uno ( está información está en un diccionario en config.py ).


- generadorTermIdDocId.py: en esta clase se realiza el proceso de las noticias. En el metodo generarTuplaTermIdDocId tomamos cada palabra del titulo y del cuerpo de cada noticia y la procesamos con un stemmer y un normalizador. Además dichas palabras no pueden ser caracteres especiales, ni tener menos de 3 caracteres. Una vez hecho esto, creamos una tupla (term_id,doc_id) en la cual el term_id es la palabra convertida a un codigo hash, el doc_id es un número formado por: numero de medio + numero de sección + id de la noticia(relleno hasta 6 dígitos). Este proceso se realiza con cada noticia, de cada medio, de cada sección y se crea un diccionario auxiliar para cada medio con clave term_id y valor lista de doc_id codificada en un bytearray, ademas de un segundo diccionario de clave palabra y valor term_id para poder decodificarlas.


- inverted_index_BSBI.py: clase encargada de crear el índice invertido tanto sin comprimir, como comprimido, utilizando el algoritmo BSBI. Toma todos los diccionarios auxiliares de cada medio, realiza un merge para formar un solo diccionario auxiliar y posteriormente crea el diccionario final con clave term_id y valor una tupla con metadata con la posición de la lista de doc_id codificada dentro de un unico string de bytes que contiene todas las listas juntas. Esto sirve para poder extraerla cuando se necesita acceder a dicha lista.


- modulo_Busqueda.py: esta clase nos permite hacer búsquedas de palabras dentro del título o cuerpo de cualquier noticia indexada, el método buscar_palabra recibe una palabra, la procesa (stemmer+normalize), y la busca dentro del diccionario de palabras:term_id. Si dicha palabra existe, la buscará en el índice invertido utilizando la tupla con metadata para extraer su lista de doc_id del string de bytes; si no existe, informará al usuario. Dicho método es utilizado por las dos funciones de búsqueda, que nos permiten buscar las noticias que incluyen cada palabra por separado, o todas las palabras en la misma noticia.


- compressed_index.py: clase encargada de la compresión de la lista de apariciones, utilizando una codificación de longitud variable. Tiene métodos de codificación/decodificación de números por separado, y de listas enteras, codificando los saltos correspondientes.


- UncompressedPostings.py: clase auxiliar para la codificación de las listas de doc_id en bytearrays.


- menu.py: menu principal del programa, da la opción de recolectar noticias, crear índices, realizar búsquedas y comprimir lista de apariciones. Implementa también el comando CTRL+C para interrumpir un proceso.



La carpeta xml incluye todas las noticias que se recopilaron hasta ahora y que se pueden seguir recopilando con la opción "1" del menu. La carpeta testFiles, incluye archivos con datos erróneos para probar los test unitarios.

Las cuatro clases de test, se encargan de probar una correcta lectura del archivo config, recoleccion y proceso de noticias y compresion de índices.

Conclusiones:

Este trabajo sirvio para profundizar temas explicados a lo largo de la cursada como manejo de XML y consultas xpath que resultaron interesantes y que pensamos que nos van a ser muy utiles mas adelante.
Ademas los puntos de indice inveritdo y compresion de lista de apariciones ,al ponerlos en practica en este TP de una manera mas compleja , nos permitio entender el funcionamiento de cada uno de una mejor manera.